In [1]:
import numpy as np
DTYPE = np.float64
import pickle
import sys
import os
import gzip
import matplotlib.pyplot as plt

In [2]:
# Flory Huggins Free Energy function
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

In [3]:
phi_globals = [
                # np.array([0.1, 0.4, 0.5], dtype=DTYPE),
                # np.array([0.1, 0.5, 0.4], dtype=DTYPE),
                # np.array([0.4, 0.1, 0.5], dtype=DTYPE),
                # np.array([0.4, 0.5, 0.1], dtype=DTYPE),
                # np.array([0.5, 0.1, 0.4], dtype=DTYPE),
                # np.array([0.5, 0.4, 0.1], dtype=DTYPE),
                # np.array([0.2, 0.3, 0.5], dtype=DTYPE),
                # np.array([0.2, 0.5, 0.3], dtype=DTYPE),
                # np.array([0.3, 0.2, 0.5], dtype=DTYPE),
                # np.array([0.3, 0.5, 0.2], dtype=DTYPE),
                # np.array([0.5, 0.2, 0.3], dtype=DTYPE),
                # np.array([0.5, 0.3, 0.2], dtype=DTYPE),
                np.array([0.2, 0.2, 0.6], dtype=DTYPE),
                # np.array([0.2, 0.6, 0.2], dtype=DTYPE),
                # np.array([0.6, 0.2, 0.2], dtype=DTYPE),
                # np.array([0.4, 0.3, 0.3], dtype=DTYPE),
                # np.array([0.3, 0.4, 0.3], dtype=DTYPE),
                # np.array([0.3, 0.3, 0.4], dtype=DTYPE),
                # np.array([0.1, 0.7, 0.2], dtype=DTYPE)
]

Xs = np.arange(0, 10.1, 0.1)
steps = 100000

In [4]:
for phi_global in phi_globals:
    F_3_phase = []
    for X in (Xs):
        # Load the distinct 3 phase solution!
        input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/"
        input_filename = f"initial_system.pkl"
        input_file = os.path.join(input_filepath, input_filename)
        
        with gzip.open(input_file, 'rb') as f:
            loaded_data = pickle.load(f)

        f = 0
        for _ in range(len(loaded_data["flory_vols"])):
            f += loaded_data["flory_vols"][_]*floryHuggins(loaded_data["flory_phases"][_], loaded_data["chis"])
        F_3_phase.append(f)        

In [5]:
initial_F_3To2 = {}
best_F_3To2 = {}

merged_compartments_list = [np.array([1, 2], dtype=np.int64), np.array([1, 3], dtype=np.int64), np.array([2, 3], dtype=np.int64)]

for merged_compartments in merged_compartments_list:
    key = tuple(merged_compartments)

    f_in = []
    f_best = []
    
    for phi_global in phi_globals:
        for X in Xs:
            input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/{merged_compartments}/"
            input_filename = f"initial_and_best.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            try:
                with gzip.open(input_file, "rb") as file:
                    loaded_data = pickle.load(file)
            
                f_in.append(loaded_data["F_initial"])
                f_best.append(loaded_data["F_best"])
            
            except FileNotFoundError:
                print(f"File not found: {input_file}, skipping...")
                continue  # if this is inside a loop
                
    initial_F_3To2[key] = f_in
    best_F_3To2[key] = f_best

In [6]:
def format_key(key):
    return '[' + ' '.join(str(x) for x in key) + ']'


In [7]:
colors = plt.cm.tab10.colors  # Uses matplotlib's default color cycle (10 colors)
fig, ax = plt.subplots(figsize=(4, 3))

ax.scatter(Xs, F_3_phase, color = "k", marker = "x", label = "3-phase", s = 5)
for ctr, (key, vals) in enumerate(best_F_3To2.items()):
    legend_label = format_key(key)
    ax.scatter(Xs, vals, label=legend_label, color=colors[ctr % len(colors)], marker = ".")
    # ax.scatter(Xs[1:], vals, label=legend_label, color=colors[ctr % len(colors)], marker = ".")

legend = ax.legend(loc='lower right', fontsize=10, frameon=True)
for text in legend.get_texts():
    text.set_fontweight("bold") 

ax.set_ylabel(r"$\mathbf{F \,(k_BT)}$", fontsize=12, fontweight="bold")
ax.set_xlabel(r"$\mathbf{X}$", fontsize=12, fontweight="bold")

ax.set_xticks(Xs[::10])

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight("bold")
                        
ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)

ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

fig.tight_layout()

output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
output_filename = f"F_3To2.png"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_file = os.path.join(output_filepath, output_filename)
plt.savefig(output_file,  dpi=400)
plt.close()

print(f"Saved @ {output_file}")

Saved @ data/withFlory/phi_g[0.2 0.2 0.6]/analysed/steps100000/F_3To2.png


In [8]:
# def are_locations_equal(loc1, loc2, tolerance=1e-2):
#     # Compare the numpy arrays [x, y, z]
#     arr_equal = np.allclose(loc1[0], loc2[0], atol=tolerance)
#     # Compare the floats [t]
#     t_equal = np.allclose(loc1[1], loc2[1], atol=tolerance)
#     return arr_equal and t_equal


def are_locations_equal(loc1, loc2, tolerance=1e-2):
    """
    Compare two locations where:
    - loc[0] is a 2D array (rows may be swapped).
    - loc[1] is a 1D array (elements may be swapped).
    """
    # Compare the 2D arrays (row order insensitive)
    arr1, arr2 = loc1[0], loc2[0]
    if arr1.shape != arr2.shape:
        return False
    # Check if all rows of arr1 exist in arr2 (within tolerance)
    row_matched = np.zeros(arr1.shape[0], dtype=bool)
    for row1 in arr1:
        for i, row2 in enumerate(arr2):
            if not row_matched[i] and np.allclose(row1, row2, atol=tolerance):
                row_matched[i] = True
                break
    arr_equal = np.all(row_matched)

    # Compare the 1D arrays (element order insensitive)
    vec1, vec2 = loc1[1], loc2[1]
    if vec1.shape != vec2.shape:
        return False
    # Check if all elements of vec1 exist in vec2 (within tolerance)
    vec_matched = np.zeros(vec1.shape[0], dtype=bool)
    for val1 in vec1:
        for i, val2 in enumerate(vec2):
            if not vec_matched[i] and np.isclose(val1, val2, atol=tolerance):
                vec_matched[i] = True
                break
    vec_equal = np.all(vec_matched)

    return arr_equal and vec_equal

In [9]:
merged_compartments_list = [np.array([1, 2], dtype=np.int64), np.array([1, 3], dtype=np.int64), np.array([2, 3], dtype=np.int64)]

for phi_global in phi_globals:
    count_distinct_mergers = []
    count_distinct_Fs = []
    for X in Xs:
        distinct_mergers = []

        # Best location for each merger
        grouped_best = []
        grouped_best_F = []
        
        for merged_compartments in merged_compartments_list:       
            input_filepath = f"data/withFlory/phi_g{phi_global}/raw/X{X:.3f}/steps{steps}/mergers/{merged_compartments}/"
            input_filename = f"initial_and_best.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            try:
                with gzip.open(input_file, "rb") as file:
                    loaded_data = pickle.load(file)
            
            except FileNotFoundError:
                print(f"File not found: {input_file}, skipping...")
                continue  # if this is inside a loop

            grouped_best.append(loaded_data["best_location"])
            # grouped_best_F.append(loaded_data["F_best"])
            grouped_best_F.append(round(loaded_data["F_best"], 4))
        count_distinct_Fs.append(len(set(grouped_best_F)))
        print(grouped_best, grouped_best_F)
        print()

        for loc in grouped_best:
            is_unique=True
            for seen_loc in distinct_mergers:
                if are_locations_equal(loc, seen_loc):
                    is_unique = False
                    break
            if is_unique:
                distinct_mergers.append(loc)
        # print(distinct_mergers)
        # print()
        count_distinct_mergers.append(len(distinct_mergers))

[[array([[0.11219254, 0.11219254, 0.77561489],
       [0.38778713, 0.38778713, 0.22442578]]), array([0.68138904, 0.31861096])], [array([[0.26843198, 0.26843198, 0.46311847],
       [0.10610498, 0.10610498, 0.78781417]]), array([0.57843057, 0.42156943])], [array([[0.09758103, 0.09758103, 0.80486123],
       [0.30600605, 0.30600605, 0.38796378]]), array([0.5086065, 0.4913935])]] [np.float64(-0.1181), np.float64(-0.1124), np.float64(-0.1142)]

[[array([[0.09747422, 0.1077891 , 0.7947377 ],
       [0.40943601, 0.38836514, 0.20219676]]), array([0.67135146, 0.32864854])], [array([[0.30920196, 0.30442587, 0.38633742],
       [0.08496143, 0.0899928 , 0.82508238]]), array([0.51301243, 0.48698757])], [array([[0.08269992, 0.08927662, 0.82798877],
       [0.36572357, 0.35643189, 0.27789357]]), array([0.58554853, 0.41445147])]] [np.float64(-0.1098), np.float64(-0.1039), np.float64(-0.1079)]

[[array([[0.4264861 , 0.38939236, 0.18412744],
       [0.08430553, 0.10325389, 0.81243757]]), array([0.33810

In [10]:
fig, ax = plt.subplots(figsize=(4, 3))

ax.scatter(Xs, count_distinct_mergers, marker = "o", s = 5)
# ax.scatter(Xs[1:], distinct_phases, marker = "x", s = 15)

ax.set_ylabel(r"Distinct Mergers", fontsize=12, fontweight="bold")
ax.set_xlabel(r"$\mathbf{X}$", fontsize=12, fontweight="bold")

ax.set_xticks(Xs[::10])
ax.set_yticks(range(0, max(count_distinct_mergers) + 2))

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight("bold")
                        
ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)

# ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

plt.tight_layout()

output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
output_filename = f"distinct_mergers_3To2_Position.png"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_file = os.path.join(output_filepath, output_filename)
plt.savefig(output_file,  dpi=400)
plt.close()

print(f"Saved @ {output_file}")

Saved @ data/withFlory/phi_g[0.2 0.2 0.6]/analysed/steps100000/distinct_mergers_3To2_Position.png


In [11]:
fig, ax = plt.subplots(figsize=(4, 3))

ax.scatter(Xs, count_distinct_Fs, marker = "o", s = 5)
# ax.scatter(Xs[1:], distinct_phases, marker = "x", s = 15)

ax.set_ylabel(r"Distinct Mergers", fontsize=12, fontweight="bold")
ax.set_xlabel(r"$\mathbf{X}$", fontsize=12, fontweight="bold")

ax.set_xticks(Xs[::10])
ax.set_yticks(range(0, max(count_distinct_Fs) + 2))

for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight("bold")
                        
ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)

# ax.grid(True, which='major', linestyle='--', linewidth=1, alpha=0.6)

plt.tight_layout()

output_filepath = f"data/withFlory/phi_g{phi_global}/analysed/steps{steps}/"
output_filename = f"distinct_mergers_3To2_Free Energy.png"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_file = os.path.join(output_filepath, output_filename)
plt.savefig(output_file,  dpi=400)
plt.close()

print(f"Saved @ {output_file}")

Saved @ data/withFlory/phi_g[0.2 0.2 0.6]/analysed/steps100000/distinct_mergers_3To2_Free Energy.png
